In [ ]:
문제 풀기: ~ 13:40

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
df = pd.read_csv('./test/sales_pos.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user       550068 non-null  int64  
 1   prod       550068 non-null  object 
 2   gender     550068 non-null  object 
 3   age_group  550068 non-null  object 
 4   job        550068 non-null  int64  
 5   city       550068 non-null  object 
 6   marital    550068 non-null  int64  
 7   prod_cat1  550068 non-null  int64  
 8   prod_cat2  376430 non-null  float64
 9   prod_cat3  166821 non-null  float64
 10  purchase   550068 non-null  int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 46.2+ MB


# Q1

In [6]:
max_prod=df.groupby('prod')['purchase'].sum().idxmax()
max_prod

'P00025442'

In [8]:
df['prod'] == max_prod

0         False
1         False
2         False
3         False
4         False
          ...  
550063    False
550064    False
550065    False
550066    False
550067    False
Name: prod, Length: 550068, dtype: bool

In [11]:
df[df['prod'] == max_prod]['job'].value_counts().idxmax()

4

In [13]:
# dir(df)

# Q2

In [15]:
df.age_group.value_counts().sort_index()

age_group
0-17      15102
18-25     99660
26-35    219587
36-45    110013
46-50     45701
51-55     38501
55+       21504
Name: count, dtype: int64

In [16]:
q2=df.copy()
cat_list=['prod_cat1', 'prod_cat2', 'prod_cat3']

q2[cat_list]=q2[cat_list].fillna(0)

In [17]:
q2[cat_list].dtypes

prod_cat1      int64
prod_cat2    float64
prod_cat3    float64
dtype: object

In [18]:
q2[cat_list].head(2)

,prod_cat1,prod_cat2,prod_cat3
0,3,0.0,0.0
1,1,6.0,14.0


In [19]:
q2[cat_list]=q2[cat_list].astype('int').astype('str')
q2[cat_list].dtypes

prod_cat1    object
prod_cat2    object
prod_cat3    object
dtype: object

In [20]:
q2[cat_list].head(2)

,prod_cat1,prod_cat2,prod_cat3
0,3,0,0
1,1,6,14


In [21]:
q2['prod_cat'] = q2['prod_cat1'] + '-' +  q2['prod_cat2'] + '-' + q2['prod_cat3']
q2.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase,prod_cat
0,1,P00069042,F,0-17,10,A,0,3,0,0,8370,3-0-0
1,1,P00248942,F,0-17,10,A,0,1,6,14,15200,1-6-14


In [22]:
q2_drop=q2.drop_duplicates(subset=['user','prod_cat'])
len(q2_drop[q2_drop.user == 1])

21

In [23]:
q2_f=q2_drop[q2_drop['age_group'] == '26-35'].reset_index(drop=True)
len(df), len(q2_f)

(550068, 85639)

In [26]:
import warnings

warnings.filterwarnings('ignore')

In [27]:
q2_m1=q2_f[q2_f.marital == 1].groupby('user').apply(len).mean()
q2_m0=q2_f[q2_f.marital == 0].groupby('user').apply(len).mean()
round(abs(q2_m1 - q2_m0),2)


0.13

# Q3

In [28]:
len(df)

550068

In [29]:
q3=df.copy()

In [30]:
q3['gender']=q3['gender'].replace({'M':1, 'F': 0})

In [33]:
gr_age=dict(pd.Series(range(7), index=df['age_group'].drop_duplicates().sort_values()))
gr_age

{'0-17': 0,
 '18-25': 1,
 '26-35': 2,
 '36-45': 3,
 '46-50': 4,
 '51-55': 5,
 '55+': 6}

In [34]:
q3['age_group'] = q3['age_group'].replace(gr_age)

In [35]:
q3['age_group'].value_counts()

age_group
2    219587
3    110013
1     99660
4     45701
5     38501
6     21504
0     15102
Name: count, dtype: int64

In [36]:
gr_list=['user','gender','age_group', 'job', 'city', 'marital']

In [38]:
q3_merge=q3.groupby(gr_list)[['prod', 'purchase']].agg({'prod':'nunique', 'purchase': 'sum'}).reset_index(drop=False)

In [40]:
q3_merge.head(2)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,0,0,10,A,0,35,334093
1,2,1,6,16,C,0,77,810472


In [41]:
len(q3_merge)

5891

In [42]:
q3_f=pd.get_dummies(q3_merge, columns=['job','city'], drop_first=False)

In [43]:
len(q3_f.columns)

30

In [44]:
q3_f=q3_f.drop(columns='user')

In [45]:
minmax=MinMaxScaler().fit(q3_f)

In [46]:
q3_m=minmax.transform(q3_f)

In [47]:
km=KMeans(n_clusters=7, random_state=123).fit(q3_m)

In [48]:
km.labels_

array([5, 2, 5, ..., 4, 3, 1])

In [ ]:
silhouette_score(